<center>
    
# Random Forest algorithm to predict calls volume in a Call Center
    
</center>

This notebook uses Call Center data from 2019. The data is for one year of call logs. The Call Center Managers want to measure its performance based on certain metrics. The managers want to rise the metrics by improving the Workforce Management (WFM) process of scheduling and staffing agents. A machine learning approach is proposed as follows:

1. Predict the daily calls volume 
2. Split the daily calls volume hourly

Random Forest algorithm is used to do the predictions.

**Importing libraries and data**

In [8]:
from utils import *

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (18,6)

In [9]:
import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

**Data Preparation**

1. The data in the files is monthly data corresponding to call center logs. The calls are registered as soon as they arrive to the call center. Aggregating by seconds or by minutes is useless since the theory of call centers suggest that the daily behavior of a call center is a Poisson model of hourly variable rate.

2. Another important thing in the theory of modeling call centers is that the calls volume and the rates are also dependent on the day of the week. The present analysis use the day of the month instead to smooth the seasonality (usually weekly) to monthly seasonality. In this way, the missing data imputation is less prompt to bias.

3. A target variable `calls volume` is created to register hourly calls volume. The numerical values of number of abandon calls, prequeue, inqueue, agent_time, postqueue, total_time and global sla achieve per hour are used as covariates (a covariate can be thought as a regressor).

In [10]:
# Load the yearly dataframe for prediction
yearly = get_yearly_frame()
yearly.sample(5, random_state = 42)

,,,abandon,prequeue,inqueue,agent_time,postqueue,total_time,sla,calls_volume
day,hour,month,,,,,,,,
8,10,8,60.0,7.730769,113.807692,137.675824,0.126374,235.098901,1.0,182.0
18,15,10,29.0,14.634328,81.955224,158.529851,0.119403,234.858209,1.0,134.0
5,17,8,29.0,9.376344,109.064516,105.290323,0.204301,192.505376,1.0,93.0
16,20,12,5.0,10.941176,63.308824,232.970588,0.161765,291.308824,1.0,68.0
17,10,1,9.0,7.889764,30.559055,206.84252,0.165354,241.590551,1.0,127.0


## **Using Random Forest**

Consider the target variable `calls_volume` with historical data of 1 year. The frequency of this variable is hourly (`H`). The covariates for this predictions will be `prequeue`, `inqueue`, `agent_time`, and `postqueue` features of the data frame.

In [11]:
from darts import TimeSeries
from darts.models import RandomForest

from darts.metrics import coefficient_of_variation, mae

Using Darts library, create the time series for the `calls_volume` target variable and the multi time series for the `features`. 

In [12]:
# Get the datafram for the year correctly formatted
df = get_formatted_time_series_frame_from(data=yearly, year="2019")

# Split Dataframe in Variables and Features
variable = df['calls_volume']
features = df[['prequeue', 'inqueue', 'agent_time', 'postqueue']] # 'total_time', 'abandon', 'sla'

# Get the darts Time Series objects
ts_variable = TimeSeries.from_series(variable,
                                     freq='H')
ts_features = TimeSeries.from_dataframe(features, 
                                        freq='H')

### **Use of Random Forest to forecast hourly calls volume - Self historical test**

As the above description suggest, the weekly seasonal behavior of 7 days impose to look back to the same amount of time in the covariates. This is the reason to select $24\times 7=168$ lags past covariates. 

In [13]:
# # Works well with 24, 48, and 168 hrs.
model_randomforest = RandomForest(lags = 168,
                                  lags_past_covariates = 168, 
                                  random_state = 42
                                  )

Train the model in the ts_variable and the ts_features

In [14]:
model_randomforest.fit(series=ts_variable,
                       past_covariates=ts_features)

KeyboardInterrupt: 

Instead of making new predictions with the model, use it to generate a historical forecast. In this sense, the performance of the algorithm is measure  for one year of data, and the validity of the results will generalize

In [ ]:
pred_series = model_randomforest.historical_forecasts(series=ts_variable,
                                                      forecast_horizon=7,
                                                      stride=1, 
                                                      retrain=False,
                                                      verbose=True,
                                                      )

**Visualizing the historical prediction**

Plot the last week of the actual data against the predicted historical data. It shows an accuracy of around 84 % according to R-RMSE metric.

In [ ]:
plot_prediction_and_test(target = ts_variable[-168:],
                         prediction= pred_series[-168:]);

### **Use of Random Forest to forecast hourly calls volume - Testing with features**

Since one week is around 2 % of a year, select 5 % of the data to be the test set and the 95% to be the training set.

In [ ]:
# Split the dataset into train / set
train_variable, test_variable = ts_variable.split_after(0.95)
train_features, test_features = ts_features.split_after(0.95)

The Random Forest model has lags of 168 periods (this mean 1 week) that matches also the past covariates lags. 

In [ ]:
# # Works well with 24, 48, and 168 hrs.
test_model = RandomForest(lags = 168,
                          lags_past_covariates = 168, 
                          random_state = 42
                         )

In [ ]:
# Train the model in the corresponding train time series
test_model.fit(series=train_variable,
               past_covariates=train_features)

In [ ]:
prediction = test_model.predict(n=len(test_variable),
                                series=train_variable,
                                past_covariates=ts_features)

**Visualizing the test prediction**

In [ ]:
plot_prediction_and_test(target=test_variable, 
                         prediction=prediction);

### **Resampling the prediction**

Scheduling staff in the call center gets improved when the aggregation time is for one shift of 8 hours. By just resampling the prediction and test variables, the R-RMSE improves only for 1 %

In [ ]:
r_train_variable = train_variable.resample(freq='8h',
                                           method='pad')
r_test_variable = test_variable.resample(freq='8h', 
                                         method='pad')
r_prediction = prediction.resample(freq='8h',
                                   method='pad')

In [ ]:
plot_prediction_and_test(target= r_test_variable, 
                         prediction=r_prediction);

**Plotting the resampled series with train set included**

In [ ]:
plot_predict(train_target=r_train_variable[-96:],
             test_target=r_test_variable,
             prediction=r_prediction,
             low_percentile=0.05);